# Tutorial 1: Land-cover classification

In this tutorial we are going to explore the power of the Google Earth Engine through classifying land-use categories on satelite imagery. 

<h2>Tutorial Outline<span class="tocSkip"></span></h2>
<hr>
<div class="toc"><ul class="toc-item">
<li><span><a href="#1.-Introducing the packages" data-toc-modified-id="1.-Introducing-the-packages-2">1. Introducing the packages</a></span></li>
<li><span><a href="#2.-Extracting and exploring Landsat-8 data" data-toc-modified-id="2.-Extracting-exploring-landsat-3">2. Extracting and exploring Landsat-8 data</a></span></li>
<li><span><a href="#3.-Create training data using Corine Land Cover" data-toc-modified-id="3.-Create-training-data-4">3. Create training data using Corine Land Cover</a></span></li>
<li><span><a href="#4.-Train the classifier using Corine Land Cover" data-toc-modified-id="4.-Classifier-CLC-5">4.-Train the classifier using Corine Land Cover</a></span></li>
<li><span><a href="#5.-Classify the Landsat-8 using Corine Land Cover" data-toc-modified-id="5.-Classify-CLC-6"> 5. Classify the Landsat-8 using Corine Land Cover</a></span></li></ul></div>

## Learning Objectives
<hr>

- .
- .
- .
- .
- .
- .

## 1. Introducing the packages
<hr>

Within this tutorial, we are going to make use of the following packages: 

[**ee**](https://developers.google.com/earth-engine/guides/python_install) is a Python package to use the the Google Earth Engine.

[**geemap**](https://geemap.org/) is a Python package for interactive mapping with the Google Earth Engine.

*We will first need to install these packages in the cell below. Uncomment them to make sure we can pip install them*

In [ ]:
#!pip install ee
#!pip install geemap

In [1]:
import ee
import geemap

## 2.Extracting and exploring the Landsat-8 data
<hr>

Unfortunately it is not as simple as just running the  code below. As soon as you run it, you will notice that you need to authorize ourselves to be able to use the Google Earth Engine

In [50]:
Map = geemap.Map(height=800,width=700,center=[52.37,4.5],zoom=7)
Map

Map(center=[52.37, 4.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Now we have succesfully managed to see a map of the Netherlands, let's add Landsat data to the map

In [51]:
point = ee.Geometry.Point([5.0, 51.37])
# point = ee.Geometry.Point([-87.7719, 41.8799])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

It worked! As we have specified that we wanted landsat data with as little clouds as possible, let's check the data and the actual cloud cover:

In [52]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-07-31'

In [53]:
image.get('CLOUD_COVER').getInfo()

0.2

## 3.Create training data using Corine Land Cover
<hr>

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [54]:
#region = Map.user_roi
#region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([4.5, 52.37]).buffer(10000)

And let's explore the Corine Land Cover data:

In [55]:
CLC = ee.Image('COPERNICUS/CORINE/V20/100m/2012').select('landcover').clip(image.geometry())
Map.addLayer(CLC, {}, 'CLC')
Map

Map(bottom=2800.0, center=[54.76964243280834, 7.670523922724533], controls=(WidgetControl(options=['position',…

Now we are going to make the training dataset.

In [39]:
points = CLC.sample(
    **{
        'region': image.geometry(),
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, 'training', False)

In [40]:
print(points.size().getInfo())

4952


In [41]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.262549916271497, 51.256151849693374]}, 'id': '0', 'properties': {'landcover': 211}}


## 4.-Train the classifier using Corine Land Cover
<hr>

In [10]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'landcover'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{'collection': points, 'properties': [label], 'scale': 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

In [11]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 371, 'B2': 504, 'B3': 935, 'B4': 839, 'B5': 4024, 'B6': 1696, 'B7': 978, 'landcover': 211}}


## 5. Classify and visualise the Landsat-8 using Corine Land Cover
<hr>

In [43]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

In [44]:
# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(bottom=2819.0, center=[54.29088164657006, 21.73095703125], controls=(WidgetControl(options=['position', 't…

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [45]:
class_values = nlcd.get('landcover_class_values').getInfo()
class_values

[111,
 112,
 121,
 122,
 123,
 124,
 131,
 132,
 133,
 141,
 142,
 211,
 212,
 213,
 221,
 222,
 223,
 231,
 241,
 242,
 243,
 244,
 311,
 312,
 313,
 321,
 322,
 323,
 324,
 331,
 332,
 333,
 334,
 335,
 411,
 412,
 421,
 422,
 423,
 511,
 512,
 521,
 522,
 523]

In [46]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

['E6004D',
 'FF0000',
 'CC4DF2',
 'CC0000',
 'E6CCCC',
 'E6CCE6',
 'A600CC',
 'A64DCC',
 'FF4DFF',
 'FFA6FF',
 'FFE6FF',
 'FFFFA8',
 'FFFF00',
 'E6E600',
 'E68000',
 'F2A64D',
 'E6A600',
 'E6E64D',
 'FFE6A6',
 'FFE64D',
 'E6CC4D',
 'F2CCA6',
 '80FF00',
 '00A600',
 '4DFF00',
 'CCF24D',
 'A6FF80',
 'A6E64D',
 'A6F200',
 'E6E6E6',
 'CCCCCC',
 'CCFFCC',
 '000000',
 'A6E6CC',
 'A6A6FF',
 '4D4DFF',
 'CCCCFF',
 'E6E6FF',
 'A6A6E6',
 '00CCF2',
 '80F2E6',
 '00FFA6',
 'A6FFE6',
 'E6F2FF']

In [47]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [49]:
Map.addLayer(landcover, {}, 'Land cover')
Map.add_legend(builtin_legend='COPERNICUS/CORINE/V20/100m')
Map

Map(bottom=2933.0, center=[51.26191485308451, 6.65771484375], controls=(WidgetControl(options=['position', 'tr…